In [1]:
from sklearn import svm
from sklearn import model_selection
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.decomposition as sklearnPCA

from ctext import *
%pylab inline
import matplotlib as mpl
import platform

if platform.system() == 'Darwin':   # I.e. if we're running on Mac OS X
    mpl.rcParams['font.family'] = 'Arial Unicode MS'
else:
    mpl.rcParams['font.family'] = "SimHei"
mpl.rcParams['font.size'] = 14

from IPython.display import display

setapikey("596b8cdac07fccbdf11559bdc41ecf6b")

def makevector(string, termlist, normalize = False):
    vector = []
    for term in termlist:
        termcount = len(re.findall(term, string))
        if normalize:
            vector.append(termcount/len(string))
        else:
            vector.append(termcount)
    return vector

Populating the interactive namespace from numpy and matplotlib


In [2]:
passages1 = gettextasobject("ctp:wb894040")#娇红记
passages2 = gettextasobject("ctp:wb70888") #西厢记
passages3 = gettextasobject("ctp:wb930815") #李娃传
passages4 = gettextasobject("ctp:wb21646") #莺莺传
passages5 = gettextasobject("ctp:wb742306") #荊釵記
yuanandbefore= passages1 + passages2 + passages3 + passages4 + passages5

In [49]:
passages1 = gettextasobject("ctp:wb842383")# 浪史奇觀 
passages2 = gettextasobject("ctp:wb728938") #牡丹亭
passages3 = gettextasobject("ctp:wb954825")#金瓶梅 
passages4 = gettextasobject("ctp:wb900066")#鼓掌绝尘
passages5 = gettextasobject("ctp:wb541826")#鐘情麗集
ming = passages1 + passages2 + passages3 + passages4 + passages5



In [47]:
passages1 = gettextasobject("ctp:hongloumeng") #红楼梦
passages2 = gettextasobject("ctp:wb369670") #镜花缘
passages3 = gettextasobject("ctp:wb944468")#再生缘
passages4 = gettextasobject("ctp:wb190138")#姑妄言
passages5 = gettextasobject("ctp:wb773438") #醒世姻緣傳
passages6 = gettextasobject("ctp:wb763355") #玉娇梨》
passages7 = gettextasobject("ctp:wb885546")#平山冷燕
qing = passages1 + passages2 + passages3 + passages4 + passages5 + passages6 + passages7


In [50]:
terms = ['有', '也', '又', '一', '不', '是', '我', '中', '了', '可', '为', '人', '两', '知', '要', '之', '在', '得', '说', '的', '他', '若', '道', '见', '且', '将', '未', '此', '这', '去', '家', '来', '上', '才', '下', '看', '著', '敢', '与', '已', '如', '能', '个', '如何', '听', '都', '便', '还', '无', '正', '就', '何', '好', '到', '把', '再', '那', '几', '甚', '却', '三', '二', '想', '虽', '出', '日', '并', '从', '你', '事', '自', '因', '既', '尽', '多', '同', '处', '先', '前', '待', '时', '但', '对', '所', '谁', '四', '小', '请', '而', '更', '过', '亦', '今日', '以', '真', '等', '向', '做', '如此', '只', '方', '各', '曾', '年', '心', '闻', '十', '则', '生', '岂', '使', '欲', '其', '倒', '送', '话', '大', '似', '于', '说道', '没', '早', '些', '里', '天', '肯', '起来', '被', '成', '即', '至', '后', '万', '非', '罢', '岁', '下回', '俱', '问', '当', '和', '行', '言', '回', '原来', '月', '如今', '叫', '每', '原', '起身', '皆', '半', '坐', '分解', '遂', '子', '故', '不过', '乃', '怎', '怕', '件', '内', '儿', '发', '就是', '知道', '长', '必', '莫', '接', '望', '会', '别', '明日', '地', '间', '一般', '这个', '意', '作', '求', '带', '觉', '十分', '快', '尚', '不知', '起', '走', '矣', '令', '入', '出来', '我们', '虽然', '段', '者', '竟', '人家', '全', '及', '许多', '少', '住', '恐', '往', '教', '首', '受', '吃', '边', '看见', '话说', '取', '遇', '最', '心中', '口', '大家', '方才', '定', '应', '随', '常', '直', '场', '寻', '怎么', '须', '放', '连', '自然', '不觉', '忙', '甚么', '这些', '晓得', '太', '用', '所以', '便是', '点', '没有','众', '深', '那里', '实', '问道', '这样', '此时', '将来', '还有', '收拾', '遍', '只怕', '转', '留', '一时', '回来', '相见', '依', '第一', '只是', '吾', '家中', '有些', '后来', '略', '正是', '或', '论', '说话', '次日', '遭', '罢了', '外', '系', '自己', '真是', '极', '君', '愿', '走到', '比', '绝', '打', '为人', '还是', '只得', '在此', '算', '另', '交', '为何', '不得', '难道', '番', '多少', '过来', '趁', '世', '众人', '耳', '果然', '白', '合', '数', '谢', '穿', '这里', '不曾', '递', '父亲', '完', '终', '且说', '一面', '听了', '任', '离', '该', '偏', '若是', '共', '观', '听见', '气', '固', '仍', '晚', '明白', '飞', '进', '正在', '由', '报', '犹', '不如', '跟', '次', '蒙', '称', '假', '旁', '兼', '反', '替', '那个', '不妨', '让', '因此', '彼此', '种', '按', '破', '叙', '吩咐', '出去', '守', '收', '凭', '身上', '勉强', '传', '打点', '初', '断', '回家', '般', '近', '第二', '这般', '当时', '东西', '迎', '惊', '到了',]
vectors1 = []
for p in yuanandbefore:
    vectors1.append(makevector(p['fulltext'], terms, True))
df1 = pd.DataFrame(vectors1, columns=terms)

vectors2 = []
for p in ming:
    vectors2.append(makevector(p['fulltext'], terms, True))
df2 = pd.DataFrame(vectors2, columns=terms)

vectors3 = []
for p in qing:
    vectors3.append(makevector(p['fulltext'], terms, True))
df3 = pd.DataFrame(vectors3, columns=terms)

In [51]:
labels = ["元及之前"] * len(df1) + ["明"] * len(df2) + ["清"] * len(df3)

df_joined = pd.concat([df1, df2, df3])
datapoints = df_joined.values.tolist() 

classifier = svm.SVC(C=10000, class_weight='balanced', kernel='linear')
classifier.fit(datapoints, labels) 

SVC(C=10000, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [52]:
scores = model_selection.cross_val_score(classifier, datapoints, labels, cv=5)
print(scores)        # Prints the score for each of 5 training and test runs
print(scores.mean()) # Prints the average score

[ 0.95419847  0.98461538  0.73643411  0.97674419  0.875     ]
0.905398430494


In [55]:
# 好逑传(test)
passages1 = gettextasobject("ctp:wb110834")

vectors1 = []
for p in passages1:
    vectors1.append(makevector(p['fulltext'], terms, True))

for n in range(0,len(vectors1)):
    prediction = classifier.predict([vectors1[n]])[0]
    print("Chapter " + str(n+1) + " " + prediction)

Chapter 1 清
Chapter 2 清
Chapter 3 清
Chapter 4 清
Chapter 5 清
Chapter 6 清
Chapter 7 清
Chapter 8 清
Chapter 9 清
Chapter 10 清
Chapter 11 清
Chapter 12 清
Chapter 13 清
Chapter 14 清
Chapter 15 清
Chapter 16 清
Chapter 17 清
Chapter 18 清


In [56]:
# 風月鑑
passages1 = gettextasobject("ctp:wb40842")
vectors1 = []
for p in passages1:
    vectors1.append(makevector(p['fulltext'], terms, True))
for n in range(0,len(vectors1)):
    prediction = classifier.predict([vectors1[n]])[0]
    print("Chapter " + str(n+1) + " " + prediction)

Chapter 1 清
Chapter 2 清
Chapter 3 清
Chapter 4 清
Chapter 5 清
Chapter 6 清
Chapter 7 清
Chapter 8 清
Chapter 9 元及之前
Chapter 10 清
Chapter 11 清
Chapter 12 清
Chapter 13 清
Chapter 14 清
Chapter 15 清
Chapter 16 清
Chapter 17 元及之前


In [58]:
# 桃花艳史
passages1 = gettextasobject("ctp:wb433726")
vectors1 = []
for p in passages1:
    vectors1.append(makevector(p['fulltext'], terms, True))
for n in range(0,len(vectors1)):
    prediction = classifier.predict([vectors1[n]])[0]
    print("Chapter " + str(n+1) + " " + prediction)

Chapter 1 清
Chapter 2 清
Chapter 3 清
Chapter 4 清
Chapter 5 清
Chapter 6 明


In [57]:
#玉燕姻緣全傳
passages1 = gettextasobject("ctp:wb546905")
vectors4 = []
for p in passages1:
    vectors1.append(makevector(p['fulltext'], terms, True))
for n in range(0,len(vectors1)):
    prediction = classifier.predict([vectors1[n]])[0]
    print("Chapter " + str(n+1) + " " + prediction)

Chapter 1 清
Chapter 2 清
Chapter 3 清
Chapter 4 清
Chapter 5 清
Chapter 6 清
Chapter 7 清
Chapter 8 清
Chapter 9 元及之前
Chapter 10 清
Chapter 11 清
Chapter 12 清
Chapter 13 清
Chapter 14 清
Chapter 15 清
Chapter 16 清
Chapter 17 元及之前
Chapter 18 清
Chapter 19 清
Chapter 20 清
Chapter 21 清
Chapter 22 清
Chapter 23 清
Chapter 24 清
